# Optimizing Bids for Keywords

Google keyword planner provides forecasts like the expected impressions, clicks, and conversions for different keywords in a **certain timeframe** depending on the daily budget **specified by the user**. This forecasts are based on **historical data, usage patterns for users and trends** exhibited across google for these keywords and enables advertisers to predict metrics in a **specified timeframe**. Also, it provides the minimum and maximum **top of page bids** required for a given keyword **i.e.** bidding **10$** for a keyword with maximum top of page bid of **8$** **does not** lead to extra clicks. We used this data to determine how much to bid for each keyword while minimizing the average Cost per Click ($CPC_{avg}$) given the specified budget. Minimizing $CPC_{avg}$ leads to better usage of the budget leading to maximizing the clicks.

By visualizing the forecasts from the keyword planner, we notice that keywords exhibit exponential growth and saturate at a certain daily budget which leads to the conclusion that after a certain point, bidding more does **not** lead to more clicks. By automating the process of setting the bids for each keyword, we achieve maximum utilization of the budget. Additionally, we can use this method for filtering out keywords that have bad conversion and suggesting ones that result in a high number of clicks.

![Google Keyword Planner](kw_planner.png "Google Keyword Planner")

## Method

We formulate the relationship between the estimated number of clicks and the daily budget. After trying different polynomials that generalize well with different keywords we define the relationship between the daily budget and clicks as follows:

$$clicks_k = b_{0k} + b_{1k} * ln(budget_k) + b_{2k} * ln(budget_k)^2$$

Then we define the objective function:

$${CPC_{avg}} = \frac{1}{K} * \sum_{k=0}^{K}{\frac{budget_k}{clicks_k}}$$

subject to:

$$budget = \sum_{k=0}^{K} budget_k = const$$
$$bid_{min}^k \leq \frac{budget_k}{clicks_k} \leq bid_{max}^k$$

where 

K: Number of keywords

bid: top of page bid

The objective function is subject to the following constraints:
1. Sum of budgets is equal to the budget specified by the user. 
2. CPC for each keyword is between the minimum and maximum top of page bids provided by the keyword planner.

We picked 4 keywords in the Weleda campaign, fitted a model for each one **from 1 to 30 June**, and set a daily budget. Then, we minimized the objective function. The minimized function sets the optimal bid for each keyword that **minimzes $CPC_{avg}$** given the daily budget.

As a sanity check, we set the budget to **27$**, and bids to the maximum top of page bid for each keyword. Then, we compared the initial CPC and clicks with the optimized ones.


$$budget = 27$$
$$bids_o = [15.34, 4.76, 5.73, 4.55]$$
$$CPC_o = 6.17$$
$$clicks_o = 186$$
$$bids_{optimal} = [1.36, 0.5, 4.75, 20.39]$$
$$CPC_{optimal} = 3.07$$
$$clicks_{optimal} = 206$$

The optimal bids result in a lower $CPC_{avg}$ and a higher number of clicks while making the budget constant. This leads to better utilization of the budget. By investigating graphs for each keyword, we notice that the $1^{st}$ keyword saturates quickly while the $4^{th}$ saturates at a higher number of clicks leading to indicating that there should be a higher allocation for the $4^{th}$ keyword than the $1^{st}$, which correctly the model does.

## Code

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge

In [2]:
kw_lst=["weleda hair tonic.csv","weleda sunscreen.csv","weleda_solskydd.csv","skinfood_weleda.csv"]
data_lst=[]

In [3]:
for i in range(len(kw_lst)):
    data = pd.read_csv(kw_lst[i])
    data["daily_budget"] = data["daily_budget"].astype(float)
    data["clicks"] = data["clicks"].astype(float)
    data_lst.append(data)

In [4]:
coef_lst=[]
interc_lst=[]
for i,x in enumerate(data_lst):
    poly=PolynomialFeatures(2)
    #xpoly=poly.fit_transform(np.power(daily_data[["daily_budget"]], 1/2.3))
    xpoly=poly.fit_transform(np.log(x[["daily_budget"]]))
    model = Ridge(alpha=1, solver='lsqr')
    model.fit(xpoly, x["clicks"].values)
    coef_lst.append(model.coef_[1:])
    interc_lst.append(model.intercept_)
    print(f"Score: {kw_lst[i][:-4]}", model.score(xpoly, x["clicks"].values))

coef_lst=np.array(coef_lst)
interc_lst=np.array(interc_lst)

Score: weleda hair tonic 0.9953490028219891
Score: weleda sunscreen 0.9636586295859639
Score: weleda_solskydd 0.994395205957459
Score: skinfood_weleda 0.9954561225600115


In [5]:
import numpy as np
import sympy as sp
from scipy.optimize import fsolve
from sympy import symbols, solve

constr_cpc= np.array([[2.94,15.34],[1.74,4.76],[2.36,5.73],[1.73,4.55]])
constr_cost=[]
min_range=0
max_range=0
#solve(x*30.4/(coef_lst[0][1]*sp.log(0) + coef_lst[0][2]*sp.log(x)**2 +interc_lst[0]) - 15.34 , x)
for i in range(len(data_lst)):
    x = symbols('x')
    func_np1 = sp.lambdify(x, x*30.4/(coef_lst[i][0]*sp.log(x) + coef_lst[i][0]*sp.log(x)**2 +interc_lst[i]) - constr_cpc[i][0], modules=['numpy'])
    solution1 = np.ceil(fsolve(func_np1, 1))
    min_range = min_range+solution1
    func_np2 = sp.lambdify(x, x*30.4/(coef_lst[i][1]*sp.log(x) + coef_lst[i][1]*sp.log(x)**2 +interc_lst[i]) - constr_cpc[i][1], modules=['numpy'])    
    solution2 = np.floor(fsolve(func_np2, 1))
    max_range = max_range+solution2
    constr_cost.append([np.round(solution1,2),np.round(solution2,2)])


In [14]:
def objective(x):
  return np.sum(x*30.4/(coef_lst[:,0]*np.log(x) + coef_lst[:,1]*np.log(x)**2 +interc_lst)) / x.shape[0]

DAILY_BUDGET = max_range  # daily sum
constr = [{'type':'eq','fun': lambda x: np.sum(x) - DAILY_BUDGET}]

def constraint_eqn(x, coef, intercpt, lim):
  return (x*30.4/(coef[0]*np.log(x) + coef[1]*np.log(x)**2 +intercpt) - lim)

for i in range(len(kw_lst)):
  
  def dummy(idx=i):
    """Dummy function to force the constraint equation to use the current value of i not the last.
    """
    constr.append({'type':'ineq', 'fun': lambda x: constraint_eqn(x[idx], coef_lst[idx], interc_lst[idx], constr_cpc[idx][0])})
    constr.append({'type':'ineq', 'fun': lambda x: -1 * constraint_eqn(x[idx], coef_lst[idx], interc_lst[idx], constr_cpc[idx][1])})
  
  dummy(i)
  
from scipy.optimize import minimize
bnds = ((0,None),(0,None),(0,None),(0,None))
init = constr_cpc[:,1]
min=minimize(objective,init,constraints=constr,options={"disp":True},bounds=bnds)

# print(min)
print(f"x0 = {init}, CPC_avg = {objective(init).round(2)}, Clicks = {np.sum((coef_lst[:,0]*np.log(init) + coef_lst[:,1]*np.log(init)**2 + interc_lst)).round(0)}")
print(f"xmin = {min['x'].round(2)}, CPC_avg = {objective(min['x']).round(2)}, Clicks = {np.sum((coef_lst[:,0]*np.log(min['x']) + coef_lst[:,1]*np.log(min['x'])**2 + interc_lst).round(0))}")

Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.0719002946551965
            Iterations: 16
            Function evaluations: 80
            Gradient evaluations: 16
x0 = [15.34  4.76  5.73  4.55], CPC_avg = 6.17, Clicks = 186.0
xmin = [ 1.36  0.5   4.75 20.39], CPC_avg = 3.07, Clicks = 206.0


In [10]:
def objective(x):
  return np.sum(-(coef_lst[:,0]*np.log(x) + coef_lst[:,1]*np.log(x)**2 +interc_lst)/(x*30.4)) / x.shape[0]

DAILY_BUDGET = max_range  # daily sum
constr = [{'type':'eq','fun': lambda x: np.sum(x) - DAILY_BUDGET}]

def constraint_eqn(x, coef, intercpt, lim):
  return (x*30.4/(coef[0]*np.log(x) + coef[1]*np.log(x)**2 +intercpt) - lim)

for i in range(len(kw_lst)):
  
  def dummy(idx=i):
    """Dummy function to force the constraint equation to use the current value of i not the last.
    """
    constr.append({'type':'ineq', 'fun': lambda x: constraint_eqn(x[idx], coef_lst[idx], interc_lst[idx], constr_cpc[idx][0])})
    constr.append({'type':'ineq', 'fun': lambda x: -1 * constraint_eqn(x[idx], coef_lst[idx], interc_lst[idx], constr_cpc[idx][1])})
  
  dummy(i)
  
from scipy.optimize import minimize
bnds = ((0,None),(0,None),(0,None),(0,None))
init = constr_cpc[:,1]
min=minimize(objective,init,constraints=constr,options={"disp":True},bounds=bnds)
print(min)
x = init
print(f"f(x0) = {np.sum((x*30.4)/(coef_lst[:,0]*np.log(x) + coef_lst[:,1]*np.log(x)**2 +interc_lst))/x.shape[0]}")
x =min['x']
print(f"f(xmin) = {np.sum((x*30.4)/(coef_lst[:,0]*np.log(x) + coef_lst[:,1]*np.log(x)**2 +interc_lst))/x.shape[0]}")


Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.41327287689194525
            Iterations: 16
            Function evaluations: 80
            Gradient evaluations: 16
     fun: -0.41327287689194525
     jac: array([0.00064931, 0.16565587, 0.018216  , 0.0189864 ])
 message: 'Optimization terminated successfully'
    nfev: 80
     nit: 16
    njev: 16
  status: 0
 success: True
       x: array([19.5832568 ,  0.4968101 ,  2.88917587,  4.03075723])
f(x0) = 6.172689589820775
f(xmin) = 4.7205538981131845
